In [1]:
!pip install torchvision

     ---------------------------------------- 1.1/1.1 MB 6.3 MB/s eta 0:00:00


In [8]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader 

from torchvision.transforms import transforms 
import torchvision.datasets as dsets



In [3]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform = transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform = transforms.ToTensor(), 
                           download=True)

100%|██████████| 9912422/9912422 [00:01<00:00, 9059053.61it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 5274566.48it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 4861725.05it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 26756360.63it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [5]:
batch_size = 100 
n_iters = 3000 

n_epochs = n_iters/ (len(train_dataset) / batch_size)
n_epochs = int(n_epochs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [30]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        
        self.maxpooling = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.fc_layer = nn.Linear(32 * 7  * 7, 10) # because of maxpooling2d(kernel_size=2)
    
    def forward(self, images):
        outputs = self.cnn1(images)
        outputs = self.relu(outputs)
        outputs = self.maxpooling(outputs) 
        
        outputs = self.cnn2(outputs)
        outputs = self.relu(outputs)
        outputs = self.maxpooling(outputs)
        
        outputs = outputs.view(outputs.size(0), -1)
        outputs = self.fc_layer(outputs)
        
        return outputs 

In [31]:
model = CNNModel()
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-2 
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [39]:
iter = 0 

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        model.train()
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct = 0 
            total = 0 
            
            model.eval()
            for images, labels in test_loader:
                images = images.requires_grad_()
                
                outputs = model(images)
                
                pred = torch.argmax(outputs.data, dim=1)
                total += labels.size(0)
                
                correct += (pred == labels).sum()
                
            accuracy = 100 * correct / total 
            
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

Iteration: 500, Loss: 0.1457, Accuracy: 95.83
Iteration: 1000, Loss: 0.1476, Accuracy: 96.45
Iteration: 1500, Loss: 0.0976, Accuracy: 96.74
Iteration: 2000, Loss: 0.1384, Accuracy: 96.99
Iteration: 2500, Loss: 0.0587, Accuracy: 97.24
Iteration: 3000, Loss: 0.0360, Accuracy: 97.65
